In [38]:
import networkx as nx
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import NEMtropy as nem
import numpy as np

In [39]:
# Load the datasets from the assignment_1_data folder
path = "World_Trade_Web/"
files = [f for f in listdir(path) if isfile(join(path, f))]
gmls = [nx.read_graphml(path + f) for f in files]

graphs = {k: v for k, v in zip(files, gmls)}

In [40]:
# Test graph from the paper
# Do not execute for the assignment

# G=nx.DiGraph()
# G.add_nodes_from(["A","B","C","D","E","F","G","H"])
# G.add_edges_from([("A","B", {"weight": 10}),("A","E", {"weight":3}), ("C", "A", {"weight":1}), ("D", "A",{"weight":2}), ("B", "G",{"weight":5}), ("B", "F",{"weight":4}), ("H", "B",{"weight":6})])
# graphs={}
# graphs["test"] = G

In [41]:
def calc_strengths(G):
    in_ = {}
    out = {}
    for source, target, weight in G.edges(data="weight"):
        if source not in out:
            out[source] = 0
        if source not in in_:
            in_[source] = 0
        if target not in out:
            out[target] = 0
        if target not in in_:
            in_[target] = 0
        if weight is not None:
            out[source] += weight
            in_[target] += weight
    return {"in": in_, "out": out}

In [42]:
def calc_s_bars_sigmas(strengths):
    W = sum(strengths["out"].values())
    s_bar_sou_out = 0
    s_bar_sou_in = 0
    s_bar_tar_in = 0
    s_bar_tar_out = 0
    for node in strengths["out"]:
        s_bar_sou_out += strengths["out"][node]*strengths["out"][node]
        s_bar_sou_in += strengths["out"][node]*strengths["in"][node]
        s_bar_tar_out += strengths["in"][node]*strengths["out"][node]
        s_bar_tar_in += strengths["in"][node]*strengths["in"][node]
    s_bar_sou_out /= W
    s_bar_sou_in /= W
    s_bar_tar_in /= W
    s_bar_tar_out /= W
    sigma_sou_out = 0
    sigma_sou_in = 0
    sigma_tar_in = 0
    sigma_tar_out = 0
    for node in strengths["out"]:
        sigma_sou_out += strengths["out"][node]*(strengths["out"][node] - s_bar_sou_out)**2
        sigma_sou_in += strengths["out"][node]*(strengths["in"][node] - s_bar_sou_in)**2
        sigma_tar_in += strengths["in"][node]*(strengths["in"][node] - s_bar_tar_in)**2
        sigma_tar_out += strengths["in"][node]*(strengths["out"][node] - s_bar_tar_out)**2
    sigma_sou_out = np.sqrt(sigma_sou_out/W)
    sigma_sou_in = np.sqrt(sigma_sou_in/W)
    sigma_tar_in = np.sqrt(sigma_tar_in/W)
    sigma_tar_out = np.sqrt(sigma_tar_out/W)
    return {"s_bar_sou_out": s_bar_sou_out, "s_bar_sou_in": s_bar_sou_in, "s_bar_tar_in": s_bar_tar_in, "s_bar_tar_out": s_bar_tar_out, "sigma_sou_out": sigma_sou_out, "sigma_sou_in": sigma_sou_in, "sigma_tar_in": sigma_tar_in, "sigma_tar_out": sigma_tar_out}

In [43]:
def calc_rhos(G, strengths, s_bars_sigmas):
    W = sum(strengths["out"].values())
    rho_out_in = 0
    rho_out_out = 0
    rho_in_in = 0
    rho_in_out = 0
    for source, target, weight in G.edges(data="weight"):
        if weight is not None:
            rho_out_in += weight*(strengths["out"][source] - s_bars_sigmas["s_bar_sou_out"])*(strengths["in"][target] - s_bars_sigmas["s_bar_tar_in"])
            rho_out_out += weight*(strengths["out"][source] - s_bars_sigmas["s_bar_sou_out"])*(strengths["out"][target] - s_bars_sigmas["s_bar_tar_out"])
            rho_in_in += weight*(strengths["in"][source] - s_bars_sigmas["s_bar_sou_in"])*(strengths["in"][target] - s_bars_sigmas["s_bar_tar_in"])
            rho_in_out += weight*(strengths["in"][source] - s_bars_sigmas["s_bar_sou_in"])*(strengths["out"][target] - s_bars_sigmas["s_bar_tar_out"])
    rho_out_in /= W*s_bars_sigmas["sigma_sou_out"]*s_bars_sigmas["sigma_tar_in"]
    rho_out_out /= W*s_bars_sigmas["sigma_sou_out"]*s_bars_sigmas["sigma_tar_out"]
    rho_in_in /= W*s_bars_sigmas["sigma_sou_in"]*s_bars_sigmas["sigma_tar_in"]
    rho_in_out /= W*s_bars_sigmas["sigma_sou_in"]*s_bars_sigmas["sigma_tar_out"]
    return {"rho_out_in": rho_out_in, "rho_out_out": rho_out_out, "rho_in_in": rho_in_in, "rho_in_out": rho_in_out}


In [44]:
strengths = {}
for k, G in graphs.items():
    strengths[k] = calc_strengths(G)

s_bars_sigmas = {}
for k, s in strengths.items():
    s_bars_sigmas[k] = calc_s_bars_sigmas(s)

rhos = {}
for k, G in graphs.items():
    rhos[k] = calc_rhos(G, strengths[k], s_bars_sigmas[k])
    print(k + " | " + "rho out-in: " + str(rhos[k]["rho_out_in"]) + "| rho out-out: " + str(rhos[k]["rho_out_out"]) + "| rho in-in: " + str(rhos[k]["rho_in_in"]) + "| rho in-out: " + str(rhos[k]["rho_in_out"]))

WDN_1992.txt.graphml | rho out-in: -0.2664117215330637| rho out-out: -0.26076773855213525| rho in-in: -0.23043052971239772| rho in-out: -0.2178041687531879
WDN_1993.txt.graphml | rho out-in: -0.2332948861604653| rho out-out: -0.21690124032555627| rho in-in: -0.23436902905204038| rho in-out: -0.2021385793483107
WDN_1994.txt.graphml | rho out-in: -0.2280788632262324| rho out-out: -0.22662914473192905| rho in-in: -0.22585786853596368| rho in-out: -0.22015233241342597
WDN_1995.txt.graphml | rho out-in: -0.21154591029146982| rho out-out: -0.20812357271965518| rho in-in: -0.21449016601768575| rho in-out: -0.2074765749509863
WDN_1996.txt.graphml | rho out-in: -0.19698652263843464| rho out-out: -0.19471663098921604| rho in-in: -0.2022276276627051| rho in-out: -0.19651218763930592
WDN_1997.txt.graphml | rho out-in: -0.1918257224497967| rho out-out: -0.19315231407495667| rho in-in: -0.1932977484020739| rho in-out: -0.19017801772442888
WDN_1998.txt.graphml | rho out-in: -0.20399322491207095| rho 

In [45]:
decms = {}
uecms = {}

for k, G in graphs.items():
    # Directed Enhanced Configuration Model
    out_degree = np.zeros(len(G))
    in_degree = np.zeros(len(G))
    n=0
    for node in G.nodes():
        out_degree[n] = G.out_degree(node)
        in_degree[n] = G.in_degree(node)
        n += 1
    out_strength = np.array(list(strengths[k]['out'].values()))
    in_strength = np.array(list(strengths[k]['in'].values()))

    # We can initialiase our graph instance with degree and strength sequence
    graph_weighted = nem.DirectedGraph(degree_sequence=np.concatenate([out_degree, in_degree]),
                                strength_sequence=np.concatenate([out_strength, in_strength]))
    graph_weighted.solve_tool(model="crema",
                          method="newton",
                          initial_guess="random",
                          adjacency="dcm_exp",
                          method_adjacency="newton")
    decms[k] = graph_weighted
    # Undirected Enhanced Configuration Model
    degree_seq = out_degree + in_degree
    strength_seq = out_strength + in_strength
    graph_weighted = nem.UndirectedGraph(degree_sequence=degree_seq, strength_sequence=strength_seq)
    graph_weighted.solve_tool(model="crema",
                          method="newton",
                          initial_guess="random",
                          adjacency="cm_exp",
                          method_adjacency="newton")
    uecms[k] = graph_weighted



c:\Users\paull\miniconda3\envs\network_science\lib\site-packages\numba\core\utils.py:613: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  category=NumbaExperimentalFeatureWarning)



solution error = 342170961427.0

solution error = 566231432964.8304

solution error = 361026877077.33167

solution error = 680206813080.2322

solution error = 406736885737.0

solution error = 786524497396.5895

solution error = 451961251474.8283

solution error = 902939880230.9824

solution error = 470661181470.2779

solution error = 936224985914.5435

solution error = 504918791947.98724

solution error = 1878726183708.4731

solution error = 499067650283.9605

solution error = 1033009244897.0477

solution error = 560139923799.0708

solution error = 1036122719337.0374

solution error = 591032103404.223

solution error = 1083865948531.3154

solution error = 271318780691243.9

solution error = 885030204288.1631

solution error = 574266627969.8245

solution error = 1061195607983.315


In [46]:
for k, cm in decms.items():
    print(k + " | DECM | error degree " + str(cm.error_degree) + " | error strength " + str(cm.error_strength))

WDN_1992.txt.graphml | DECM | error degree 2.064552973024547e-09 | error strength 342170961427.0
WDN_1993.txt.graphml | DECM | error degree 7.360011267110167e-09 | error strength 361026877077.33167
WDN_1994.txt.graphml | DECM | error degree 3.825643801746992e-09 | error strength 406736885737.0
WDN_1995.txt.graphml | DECM | error degree 2.5713156048823294e-09 | error strength 451961251474.8283
WDN_1996.txt.graphml | DECM | error degree 1.8550743163814332e-09 | error strength 470661181470.2779
WDN_1997.txt.graphml | DECM | error degree 2.9827731395926094e-09 | error strength 504918791947.98724
WDN_1998.txt.graphml | DECM | error degree 3.5130653941450873e-09 | error strength 499067650283.9605
WDN_1999.txt.graphml | DECM | error degree 3.336111831231392e-09 | error strength 560139923799.0708
WDN_2000.txt.graphml | DECM | error degree 2.7111468625662383e-09 | error strength 591032103404.223
WDN_2001.txt.graphml | DECM | error degree 2.364767937024226e-09 | error strength 271318780691243.9


In [47]:
for k, cm in uecms.items():
    print(k + " | UECM | error degree " + str(cm.error_degree) + " | error strength " + str(cm.error_strength))

WDN_1992.txt.graphml | UECM | error degree 50.0 | error strength 566231432964.8304
WDN_1993.txt.graphml | UECM | error degree 111.82500824831705 | error strength 680206813080.2322
WDN_1994.txt.graphml | UECM | error degree 104.25956942651635 | error strength 786524497396.5895
WDN_1995.txt.graphml | UECM | error degree 101.0 | error strength 902939880230.9824
WDN_1996.txt.graphml | UECM | error degree 108.0 | error strength 936224985914.5435
WDN_1997.txt.graphml | UECM | error degree 116.00000051610294 | error strength 1878726183708.4731
WDN_1998.txt.graphml | UECM | error degree 118.0 | error strength 1033009244897.0477
WDN_1999.txt.graphml | UECM | error degree 126.85862465391688 | error strength 1036122719337.0374
WDN_2000.txt.graphml | UECM | error degree 125.0 | error strength 1083865948531.3154
WDN_2001.txt.graphml | UECM | error degree 127.00000005304537 | error strength 885030204288.1631
WDN_2002.txt.graphml | UECM | error degree 120.0 | error strength 1061195607983.315


In [48]:
sample_networks = {}
# Load the datasets from the assignment_1_data folder
path = "samples_"

for k, cm in decms.items():
    cm.ensemble_sampler(n=30, output_dir=path + k + "/")
    sample_networks[k] = []
    files = [f for f in listdir(path + k) if isfile(join(path + k, f))]
    edgelist_ens = [np.loadtxt(path + k + "/" + f) for f in files]

    for i, e in enumerate(edgelist_ens):
        ens_adj = nem.network_functions.build_adjacency_from_edgelist(edgelist = e,
                                                is_directed = True,
                                                is_sparse = False,
                                                is_weighted = True)
        sample_networks[k].append(nx.from_numpy_array(ens_adj))

In [49]:
sample_strengths = {}
for k, s in sample_networks.items():
    sample_strengths[k] = [calc_strengths(G) for G in s]

sample_s_bars_sigmas = {}
for k, ss in sample_strengths.items():
    sample_s_bars_sigmas[k] = [calc_s_bars_sigmas(s) for s in ss]

rhos = {}
for k, Gs in sample_networks.items():
    rhos[k] = []
    for i in range(len(Gs)):
        rhos[k].append(calc_rhos(Gs[i], sample_strengths[k][i], sample_s_bars_sigmas[k][i]))

In [50]:
average_rhos = {}

for k, r in rhos.items():
    average_rhos[k] = {}
    for i in range(len(r)):
        for key, value in r[i].items():
            if key not in average_rhos[k]:
                average_rhos[k][key] = []
            average_rhos[k][key].append(value)
            

In [51]:
print("Average rhos")
for k, r in average_rhos.items():
    print(k)
    for key, value in r.items():
        print(key + ": " + str(np.mean(value)) + " +- " + str(np.std(value)))

Average rhos
WDN_1992.txt.graphml
rho_out_in: -0.4524136097462731 +- 0.2912954008636417
rho_out_out: -0.020546905003460254 +- 0.23582223921870338
rho_in_in: -0.28308736901821063 +- 0.17241043189652525
rho_in_out: -0.21734764652132738 +- 0.1390198699981063
WDN_1993.txt.graphml
rho_out_in: 0.6879958343249734 +- 0.22679287984427335
rho_out_out: -0.3917788125653896 +- 0.30017971629607637
rho_in_in: 0.3553505377050953 +- 0.42508770476197033
rho_in_out: -0.40592235952443073 +- 0.2597670032084101
WDN_1994.txt.graphml
rho_out_in: 0.41272481145721696 +- 0.3272423792397845
rho_out_out: 0.11775145586661567 +- 0.2028685816365619
rho_in_in: 0.005492200948734187 +- 0.19574285851314074
rho_in_out: -0.11030235609848606 +- 0.09232642142972228
WDN_1995.txt.graphml
rho_out_in: 0.513720201827475 +- 0.2985298499088806
rho_out_out: 0.07948505740001076 +- 0.5746455445572516
rho_in_in: -0.1514902613895401 +- 0.3639768438069927
rho_in_out: -0.45011678645028486 +- 0.16974174374270795
WDN_1996.txt.graphml
rho_ou